### FLiBe Simple

In [1]:
import openmc
import numpy as np
import matplotlib.pyplot as plt

#### Nuclear Constants & Helper functions

In [13]:
# Nuclear constants
#   from atom.kaeri.re.kr/nuchart/
AVO = 6.022141e+23
LI6_AMU, LI7_AMU = 6.015122, 7.016004 # amu


# Helper functions
def li_wtfracs(w6):
    # Convert a lithium-6 enrichment by weight fraction into atomic fractions of Li-6 and Li-7.
    # 
    # Parameters: w6 : float : Weight fraction of Li-6 (ex. 0.20 for 20 wt% Li-6)
    # Returns: x6, x7 : float, float : Atomic fractions of Li-6, Li-7

    if not 0.0 <= w6 <= 1.0: 
        raise ValueError("li6_enrich must be between 0 and 1")

    w7 = 1.0 - w6 # wt frac Li-7
    n6, n7 = w6 / LI6_AMU, w7 / LI7_AMU 
    x6, x7 = n6 / (n6 + n7), n7 / (n6 + n7) # Atomic fractions

    return x6, x7

In [6]:
# User specifications
FLIBE_DENSITY = 1.9 # usu 1.8-2.0 g/cc
LI_ENRICHMENT = 0.2

# Materials definitions
flibe = openmc.Material(name="FLiBe")
flibe.set_density('g/cm3', FLIBE_DENSITY) # usu 1.8-2.0 g/cc

N_li6, N_li7 = li_wtfracs(LI_ENRICHMENT)
flibe.add_nuclide('Li6', N_li6,)
flibe.add_nuclide('Li6', N_li6,)

flibe.add_nuclide('Li6', N_li6,)
flibe.add_nuclide('Li6', N_li6,)

materials = openmc.Materials([flibe])
materials.export_to_xml("./xml/materials.xml")

#### Geometry
For our simple case suppose we make a 1 cm$^3$ cube with reflective surfaces of homogeneous FLiBe.

In [10]:
# Geometry definitions
xmin = openmc.XPlane(-0.5, boundary_type='reflective')
xmax = openmc.XPlane( 0.5, boundary_type='reflective')
ymin = openmc.YPlane(-0.5, boundary_type='reflective')
ymax = openmc.YPlane( 0.5, boundary_type='reflective')
zmin = openmc.ZPlane(-0.5, boundary_type='reflective')
zmax = openmc.ZPlane( 0.5, boundary_type='reflective')

region = +xmin & -xmax & +ymin & -ymax & +zmin & -zmax
cell = openmc.Cell(fill=flibe, region=region)
root_univ = openmc.Universe(cells=[cell])

geometry = openmc.Geometry(root_univ)
geometry.export_to_xml("./xml/geometry.xml")

In [9]:
# Problem settings - Tallies
tallies = openmc.Tallies()

# (n,g) MT=102 tallies for each isotope in the system
for nuc in ['Li6', 'Li7', 'Be9', 'F19']:
    filt = openmc.NuclideFilter(nuc)
    tally = openmc.Tally(name=f"capture_{nuc}")
    tally.filters = [filt]
    tally.scores = ['(n,g)']
    tallies.append(tally)

AttributeError: module 'openmc' has no attribute 'NuclideFilter'

Run OpenMC

In [12]:
# if __name__ == '__main__':
#     # If you were to make this a .py script then you should place everything below underneath this if statement
openmc.run()
# sp = openmc.Statepoint(f"statepoint.{settings.batches}.h5")

 ERROR: Could not find any XML input files! In order to run OpenMC, you first
        need a set of input files; at a minimum, this includes settings.xml,
        geometry.xml, and materials.xml or a single model XML file. Please
        consult the user's guide at https://docs.openmc.org for further
        information.


RuntimeError: Could not find any XML input files! In order to run OpenMC, you first need a set of input files; at a minimum, this includes settings.xml, geometry.xml, and materials.xml or a single model XML file. Please consult the user's guide at https://docs.openmc.org for further information.